## Imports

In [1]:
from sklearn.naive_bayes import MultinomialNB
import pandas as pd
import numpy as np
from pandarallel import pandarallel
from sklearn.metrics import confusion_matrix,classification_report
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.feature_extraction.text import HashingVectorizer, CountVectorizer
from sklearn.pipeline import Pipeline
import nltk
import re
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
from nltk.stem import WordNetLemmatizer
nltk.download('punkt')
from sklearn.model_selection import learning_curve
import matplotlib.pyplot as plt
import time
print('imports done.')

[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\user\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!


imports done.


## Data loading and pre-processing

In [2]:
movie_reviews_dataset = pd.read_json('../dataset_management/dataset_with_keyword_counts.json')

# Transpose dataset to put IDs as rows and have 5 columns instead.
movie_reviews_dataset = movie_reviews_dataset.T

movie_reviews_dataset.head()

,review,rating,label,positive_keywords,negative_keywords
3fbe21c4-3046-4b12-abc0-77ebdfaf09b1,I was completely mesmerized by Lekin and espec...,9/10,Positive,38,12
3ff18551-e522-4de2-ba9d-0d11387e62ba,Greatly enjoyed the development of the story l...,9/10,Positive,30,7
ee8306d6-acc4-4b8f-a333-2c86552de3ea,"In Lekin, Sameer, a museum curator working for...",9/10,Positive,69,33
9edd5843-c446-4985-80c7-dbfe0b90dd1b,"The lines of time are very blurry. Past, prese...",10/10,Positive,14,10
cf960381-1782-40a0-a870-435d9f636d09,Yaara Sili Sili Virah Ki Raat Ka Jalna'Lekin' ...,8/10,Positive,57,20


In [3]:
movie_reviews_dataset['review'].iloc[0]

"I was completely mesmerized by Lekin and especially by the castle and Dimple's haunting character who took over the screen whenever she materialized. It brought back memories of Ugetsu, one of my favorite Japanese films. The photography, the music, story, acting was all top notch and I am definitely going to have to see what else Gulzar did. The desert, itself, was as much a character as any of the actors. I would have given Lekin a 10 but, since I do not know Hindi, I have to rely on subtitles and, periodically, the subtitles were a little hard to follow unless you are a speed reader. Also, my biggest gripe with a lot of great Hindi films is that they do not include the subtitles to the songs which was the case on my DVD. I was unable to see for myself why Gulzar won two awards for best lyrics that year. It was also frustrating because, when Dimple's character sang, I am almost certain that knowing the lyrics would have only added to the enjoyment of the film. I still gave Lekin a 9 

In [4]:
len(movie_reviews_dataset)

14998

In [5]:
# Check for NaN values and remove them
print("NaN (before cleanup) ?: \n", movie_reviews_dataset.isnull().sum())

movie_reviews_dataset['review'] = movie_reviews_dataset['review'].fillna('')
movie_reviews_dataset['positive_keywords'] = movie_reviews_dataset['positive_keywords'].fillna(0)
movie_reviews_dataset['negative_keywords'] = movie_reviews_dataset['negative_keywords'].fillna(0)

print("NaN (after cleanup) ?: \n", movie_reviews_dataset.isnull().sum())

NaN (before cleanup) ?: 
 review                0
rating                0
label                 0
positive_keywords     3
negative_keywords    24
dtype: int64
NaN (after cleanup) ?: 
 review               0
rating               0
label                0
positive_keywords    0
negative_keywords    0
dtype: int64


In [6]:
# Text preprocessing for review column
# Lower all text

movie_reviews_dataset['review'] = movie_reviews_dataset['review'].str.lower()

# Initialize pandarallel
# I used pandarallel because it applies the functions much faster than a normal pandas apply.
pandarallel.initialize(nb_workers=4,progress_bar=True)

# Remove all special characters
def remove_special_chars(text):
    return ''.join(x if x.isalnum() else ' ' for x in text)

movie_reviews_dataset['review'] = movie_reviews_dataset['review'].parallel_apply(remove_special_chars)

# get stopwords.
stop_words = set(stopwords.words('english'))

# Remove stop_words
def remove_stopwords(text):
    words = word_tokenize(text)
    return [x for x in words if x not in stop_words]


movie_reviews_dataset['review'] = movie_reviews_dataset['review'].apply(remove_stopwords)

# Lemmatization
def lemmatize_word(text):
    wordnet = WordNetLemmatizer()
    return " ".join([wordnet.lemmatize(word) for word in text])


movie_reviews_dataset['review'] = movie_reviews_dataset['review'].apply(lemmatize_word)

# Remove numbers
def remove_numbers(text):
    return re.sub(r'\d+', '', text)

movie_reviews_dataset['review'] = movie_reviews_dataset['review'].apply(remove_numbers)

print('Example of preprocessing train: ')
print(movie_reviews_dataset['review'].iloc[0])

INFO: Pandarallel will run on 4 workers.
INFO: Pandarallel will use standard multiprocessing data transfer (pipe) to transfer data between the main process and workers.

https://nalepae.github.io/pandarallel/troubleshooting/


Example of preprocessing train: 
completely mesmerized lekin especially castle dimple haunting character took screen whenever materialized brought back memory ugetsu one favorite japanese film photography music story acting top notch definitely going see else gulzar desert much character actor would given lekin  since know hindi rely subtitle periodically subtitle little hard follow unless speed reader also biggest gripe lot great hindi film include subtitle song case dvd unable see gulzar two award best lyric year also frustrating dimple character sang almost certain knowing lyric would added enjoyment film still gave lekin  anyone tell easily find lyric would grateful


## Model creation and evaluation

In [33]:
X = movie_reviews_dataset[['review','positive_keywords', 'negative_keywords']]
y = movie_reviews_dataset['label']
vectorizer = CountVectorizer()
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.5, random_state=42)

train_text = X_train['review']
train_text = vectorizer.fit_transform(train_text)
X_train['review'] = train_text.shape[0]

test_text = X_test['review']
test_text = vectorizer.transform(test_text)
X_test['review'] = test_text.shape[0]

print('data vectorization complete.')

data vectorization complete.


In [34]:
naive_bayes = MultinomialNB()
naive_bayes.fit(X_train, y_train)

MultinomialNB()

In [35]:
y_pred = naive_bayes.predict(X_test)

cnf_matrix = confusion_matrix(y_test, y_pred)
print("\nConfusion Matrix:\n", cnf_matrix)

print("\nClassification Report:\n", classification_report(y_test, y_pred))


Confusion Matrix:
 [[   0  323 1085]
 [   0  414 1070]
 [   0 1047 3560]]

Classification Report:
               precision    recall  f1-score   support

    Negative       0.00      0.00      0.00      1408
     Neutral       0.23      0.28      0.25      1484
    Positive       0.62      0.77      0.69      4607

    accuracy                           0.53      7499
   macro avg       0.28      0.35      0.31      7499
weighted avg       0.43      0.53      0.47      7499



C:\Users\user\anaconda3\Lib\site-packages\sklearn\metrics\_classification.py:1469: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
C:\Users\user\anaconda3\Lib\site-packages\sklearn\metrics\_classification.py:1469: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
C:\Users\user\anaconda3\Lib\site-packages\sklearn\metrics\_classification.py:1469: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


### Best hyperparam test

In [36]:
# Using best hyperparams from previous notebook
best_nb = MultinomialNB(alpha=1.0)
best_nb.fit(X_train, y_train)

y_pred = best_nb.predict(X_test)

cnf_matrix = confusion_matrix(y_test, y_pred)
print("\nConfusion Matrix:\n", cnf_matrix)

print("\nClassification Report:\n", classification_report(y_test, y_pred))


Confusion Matrix:
 [[   0  323 1085]
 [   0  414 1070]
 [   0 1047 3560]]

Classification Report:
               precision    recall  f1-score   support

    Negative       0.00      0.00      0.00      1408
     Neutral       0.23      0.28      0.25      1484
    Positive       0.62      0.77      0.69      4607

    accuracy                           0.53      7499
   macro avg       0.28      0.35      0.31      7499
weighted avg       0.43      0.53      0.47      7499



C:\Users\user\anaconda3\Lib\site-packages\sklearn\metrics\_classification.py:1469: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
C:\Users\user\anaconda3\Lib\site-packages\sklearn\metrics\_classification.py:1469: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
C:\Users\user\anaconda3\Lib\site-packages\sklearn\metrics\_classification.py:1469: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


Elapsed Time for learning curve creation: 0.7006113529205322 seconds


In [29]:
# Learning curve
start_time = time.time()
def plot_learning_curve(X, y, estimator, train_sizes=np.linspace(0.1, 1.0, 10), cv=5):
    train_sizes, train_scores, test_scores = learning_curve(estimator, X, y, train_sizes=train_sizes, cv=cv)

    train_scores_mean = np.mean(train_scores, axis=1)
    test_scores_mean = np.mean(test_scores, axis=1)

    plt.plot(train_sizes, train_scores_mean, label='Training score')
    plt.plot(train_sizes, test_scores_mean, label='Testing score')

    plt.title('Learning Curve')
    plt.xlabel('Training Size')
    plt.ylabel('Score')
    plt.legend()
    plt.show()
    
plot_learning_curve(X_test, y_test, best_nb)
end_time = time.time()
elapsed_time = end_time - start_time
print("Elapsed Time for learning curve creation:", elapsed_time, "seconds")